In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np

from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *

from utils_motor_sigproc import normalize_band
from utils_motor_plot import draw_CS_boundary, plot_band_power

ROOT_LOAD_DIR = f'{MODEL_INPUT_DIR}/scalogram_matrix'
# ROOT_SAVE_DIR = f'{MODEL_INPUT_DIR}/plots_and_movies'
ROOT_SAVE_DIR = '.'

In [ ]:
""" channel band mean and standard deviation """
spect_mu  = np.load(f'{BAND_MU_SIGMA_DIR}/spect_mu.npy')
lmp_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_mu.npy')
lfs_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_mu.npy')
beta_mu   = np.load(f'{BAND_MU_SIGMA_DIR}/beta_mu.npy')
lga_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/lga_mu.npy')
hga_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/hga_mu.npy')

spect_sigma  = np.load(f'{BAND_MU_SIGMA_DIR}/spect_sigma.npy')
lmp_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_sigma.npy')
lfs_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_sigma.npy')
beta_sigma   = np.load(f'{BAND_MU_SIGMA_DIR}/beta_sigma.npy')
lga_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/lga_sigma.npy')
hga_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/hga_sigma.npy')

Generate Movie

In [ ]:
""" load data """
key = '010'
load_dir = f'{SPECT_DATA_DIR}/{key}'
assert os.path.exists(load_dir)

# recording data
good_chs    = np.load(f'{load_dir}/good_channels_{key}.npy')
spect       = np.load(f'{load_dir}/spect_{key}.npy'     ) 
lmp_data    = np.load(f'{load_dir}/lmp_{key}.npy'       ) 
spect_t     = np.load(f'{load_dir}/spect_t_{key}.npy'   ) 

spect_f     = np.load(f'{load_dir}/spect_f_{key}.npy'   ) 
beta_fidxs  = np.load(f'{load_dir}/beta_fidxs_{key}.npy') 
lga_fidxs   = np.load(f'{load_dir}/lga_fidxs_{key}.npy' ) 
hga_fidxs   = np.load(f'{load_dir}/hga_fidxs_{key}.npy' ) 

# motion data
wrist_vel_x = np.load(f'{MODEL_INPUT_DIR}/motion/{key}/norm_wrist_vel_x_{key}.npy')
wrist_vel_y = np.load(f'{MODEL_INPUT_DIR}/motion/{key}/norm_wrist_vel_y_{key}.npy')
wrist_vel_z = np.load(f'{MODEL_INPUT_DIR}/motion/{key}/norm_wrist_vel_z_{key}.npy')
model_t     = np.load(f'{MODEL_INPUT_DIR}/motion/{key}/model_t_{key}.npy')

""" extract bands """
spect_beta = np.sum(spect[:,:,beta_fidxs], axis=2)
spect_hga  = np.sum(spect[:,:, hga_fidxs], axis=2)

""" z-score """
zs_lmp = normalize_band(good_chs, lmp_data, lmp_mu, lmp_sigma, sel_zscore=True)
zs_beta = normalize_band(good_chs, spect_beta, beta_mu, beta_sigma, sel_zscore=True)
zs_hga = normalize_band(good_chs, spect_hga, hga_mu, hga_sigma, sel_zscore=True)

idx0 = np.where(spect_t >= model_t[0])[0][0]
idx1 = np.where(spect_t <= model_t[-1])[0][-1] + 1

In [ ]:
""" pad the band data to full 16x16 channels and truncate in time domain """
padded_zs_lmp  = np.zeros((NCH, idx1-idx0))
padded_zs_beta = np.zeros((NCH, idx1-idx0))
padded_zs_hga  = np.zeros((NCH, idx1-idx0))

padded_zs_lmp[good_chs]  = zs_lmp [:,idx0:idx1]
padded_zs_beta[good_chs] = zs_beta[:,idx0:idx1]
padded_zs_hga[good_chs]  = zs_hga [:,idx0:idx1]

assert wrist_vel_y.shape[0] == padded_zs_lmp.shape[1]

In [ ]:
""" decide on quantile to apply for colormap range """
q_cut = 0.005
vmin_lmp  = np.quantile(zs_lmp,  q_cut)
vmax_lmp  = np.quantile(zs_lmp,  1-q_cut)
vmin_beta = np.quantile(zs_beta, q_cut)
vmax_beta = np.quantile(zs_beta, 1-q_cut)
vmin_hga  = np.quantile(zs_hga,  q_cut)
vmax_hga  = np.quantile(zs_hga,  1-q_cut)

assert vmin_lmp < 0
abs_vmax_lmp = max(-vmin_lmp, vmax_lmp)
vmin_lmp = -1*abs_vmax_lmp
vmax_lmp = abs_vmax_lmp

In [ ]:
""" labels and colormap  """
title_str0 = 'LMP (z-scored)'
title_str1 = 'β (z-scored)'
title_str2 = 'High γ (z-scored)'
cmap0, cmap1, cmap2 = 'bwr', 'viridis', 'viridis'

In [ ]:
""" define time segment to create into a movie """
# idx0, idx1 are overwritten..
model_t -= model_t[0]
t_start, t_end = 0, 20

idx0 = np.where(model_t >= t_start)[0][0]
idx1 = np.where(model_t <= t_end)[0][-1] + 1

In [ ]:
""" plot initial frame """
fig, ax = plt.subplots(2, 3, figsize=(10, 6.5),
                   gridspec_kw={'height_ratios': [1, 3]}, sharey='row')

plt.subplots_adjust(left=0.1, bottom=0.03, right=0.9, top=0.85,
                    wspace=0.1, hspace=0.05)

# step 1. plot motor feature and add vlines
line0 = ax[0,0].axvline(x = t_start, color='k', linestyle='--', linewidth=2)
line1 = ax[0,1].axvline(x = t_start, color='k', linestyle='--', linewidth=2)
line2 = ax[0,2].axvline(x = t_start, color='k', linestyle='--', linewidth=2)

ax[0,0].plot(model_t, wrist_vel_x)
ax[0,1].plot(model_t, wrist_vel_y)
ax[0,2].plot(model_t, wrist_vel_z)

# step 2. plot bands as 16x16 frames
idx = 0
frame0 = padded_zs_lmp [:,idx].reshape(16, -1)
frame1 = padded_zs_beta[:,idx].reshape(16, -1)
frame2 = padded_zs_hga [:,idx].reshape(16, -1)

im0 = ax[1,0].imshow(frame0, vmin=vmin_lmp,  vmax=vmax_lmp,  cmap=cmap0)
im1 = ax[1,1].imshow(frame1, vmin=vmin_beta, vmax=vmax_beta, cmap=cmap1)
im2 = ax[1,2].imshow(frame2, vmin=vmin_hga,  vmax=vmax_hga,  cmap=cmap2)


# step 3. add labels and title
for c in range(3):
    ax[0,c].set_yticks([0])
    ax[0,c].set_xlim((t_start, t_end))
    ax[0,c].set_xlabel('Time (s)')

    draw_CS_boundary(ax[1,c])
    ax[1,c].set_xticks([])
    ax[1,c].set_yticks([])

ax[0,0].set_title('wrist x-velocity\n(normalized)')
ax[0,1].set_title('wrist y-velocity\n(normalized)')
ax[0,2].set_title('wrist z-velocity\n(normalized)')

ax[1,0].set_title(title_str0)
ax[1,1].set_title(title_str1)
ax[1,2].set_title(title_str2)

# add colorbar
bottom_cbar = 0.10  # Adjust bottom position of the colorbars
width_cbar = 0.25  # Adjust width of the colorbars
height_cbar = 0.025 # Adjust height of the colorbars

left_cbar0 = 0.1  # Adjust left position of the colorbars
left_cbar1 = 0.375  # Adjust left position of the colorbars
left_cbar2 = 0.65  # Adjust left position of the colorbars

cbar0_ax = fig.add_axes([left_cbar0, bottom_cbar, width_cbar, height_cbar])
cbar1_ax = fig.add_axes([left_cbar1, bottom_cbar, width_cbar, height_cbar])
cbar2_ax = fig.add_axes([left_cbar2, bottom_cbar, width_cbar, height_cbar])
fig.colorbar(im0, ax=ax[1, 0], orientation='horizontal', cax=cbar0_ax)
fig.colorbar(im1, ax=ax[1, 1], orientation='horizontal', cax=cbar1_ax)
fig.colorbar(im2, ax=ax[1, 2], orientation='horizontal', cax=cbar2_ax) 

In [ ]:
""" generate movie """
# download FFMPegWriter from here: https://ffmpeg.org/download.html
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter

def animate(idx):
    suptitle_str = f't={model_t[idx]:.2f} s'
    fig.suptitle(suptitle_str, y = 0.99)
    if suptitle_str[-4:][:2] == '00':
        print(suptitle_str)

    line0.set_xdata(model_t[idx])
    line1.set_xdata(model_t[idx])
    line2.set_xdata(model_t[idx])

    frame0 = padded_zs_lmp [:,idx].reshape(16, -1)
    frame1 = padded_zs_beta[:,idx].reshape(16, -1)
    frame2 = padded_zs_hga [:,idx].reshape(16, -1)

    im0.set_array(frame0)
    im1.set_array(frame1)
    im2.set_array(frame2)

    return line0, line1, line2, im0, im1, im2

frames = np.arange(idx0, idx1, 2)
ani = animation.FuncAnimation(fig, animate, frames=frames, blit=True)

save_fn = f'bisc_nhp_motor_session_{key}'
ani.save(f'{ROOT_SAVE_DIR}/{save_fn}.mp4', writer=FFMpegWriter(fps=10))